In [1]:
%load_ext autoreload
%autoreload 2
from cyecca import lie
import casadi as ca
import numpy as np
import scipy.linalg

In [2]:
np.set_printoptions(precision=3, linewidth=100)

In [8]:
x = lie.se23.elem(ca.vertcat(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.1, 0.2, 0.3))

$e^{ad_x} = Ad e^x$

In [9]:
exp_ad_x = scipy.linalg.expm(ca.DM(x.ad()))
exp_ad_x

array([[ 0.936, -0.283,  0.21 ,  0.   ,  0.   ,  0.   , -0.127, -0.26 ,  0.215],
       [ 0.303,  0.951, -0.068,  0.   ,  0.   ,  0.   ,  0.299, -0.098, -0.034],
       [-0.181,  0.127,  0.975,  0.   ,  0.   ,  0.   , -0.157,  0.152, -0.049],
       [ 0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 , -0.273, -0.491,  0.556],
       [ 0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068,  0.618, -0.215, -0.248],
       [ 0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975, -0.379,  0.512, -0.137],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975]])

In [10]:
Ad_exp_x = np.array(ca.DM(x.exp(lie.SE23Mrp).Ad()))
Ad_exp_x

array([[ 0.936, -0.283,  0.21 ,  0.   ,  0.   ,  0.   , -0.127, -0.26 ,  0.215],
       [ 0.303,  0.951, -0.068,  0.   ,  0.   ,  0.   ,  0.299, -0.098, -0.034],
       [-0.181,  0.127,  0.975,  0.   ,  0.   ,  0.   , -0.157,  0.152, -0.049],
       [ 0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 , -0.273, -0.491,  0.556],
       [ 0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068,  0.618, -0.215, -0.248],
       [ 0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975, -0.379,  0.512, -0.137],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975]])

In [11]:
np.linalg.norm(exp_ad_x - Ad_exp_x)

3.1409514705139295e-16

In [3]:
x.algebra.elem(ca.SX.sym("x", 9)).ad()

SX(@1=(-x_7), @2=(-x_8), @3=(-x_6), 
[[00, @2, x_7, 00, 00, 00, 00, (-x_5), x_4], 
 [x_8, 00, @3, 00, 00, 00, x_5, 00, (-x_3)], 
 [@1, x_6, 00, 00, 00, 00, (-x_4), x_3, 00], 
 [00, 00, 00, 00, @2, x_7, 00, (-x_2), x_1], 
 [00, 00, 00, x_8, 00, @3, x_2, 00, (-x_0)], 
 [00, 00, 00, @1, x_6, 00, (-x_1), x_0, 00], 
 [00, 00, 00, 00, 00, 00, 00, @2, x_7], 
 [00, 00, 00, 00, 00, 00, x_8, 00, @3], 
 [00, 00, 00, 00, 00, 00, @1, x_6, 00]])

In [4]:
x.left_jacobian()

SX(@1=0.978484, @2=0.151568, @3=-0.0938736, @4=-0.144948, @5=0.98345, @6=0.0593496, @7=0.103804, @8=-0.0394891, @9=0.991725, 
[[@1, @4, @7, 00, 00, 00, -0.0427297, -0.138217, 0.106388], 
 [@2, @5, @8, 00, 00, 00, 0.151364, -0.032869, -0.0285422], 
 [@3, @6, @9, 00, 00, 00, -0.0866664, 0.067985, -0.0164345], 
 [00, 00, 00, @1, @4, @7, -0.0919934, -0.267181, 0.271445], 
 [00, 00, 00, @2, @5, @8, 0.31, -0.0722919, -0.150667], 
 [00, 00, 00, @3, @6, @9, -0.212181, 0.239404, -0.0460762], 
 [00, 00, 00, 00, 00, 00, @1, @4, @7], 
 [00, 00, 00, 00, 00, 00, @2, @5, @8], 
 [00, 00, 00, 00, 00, 00, @3, @6, @9]])

In [15]:
np.array(
    ca.DM(x.right_jacobian()) @ scipy.linalg.expm(ca.DM(x.ad()))
    - ca.DM(x.left_jacobian())
)

array([[ 0.000e+00,  1.665e-16, -1.249e-16,  0.000e+00,  0.000e+00,  0.000e+00,  2.984e-16,
         1.332e-15, -9.021e-16],
       [-1.110e-16,  0.000e+00,  4.163e-17,  0.000e+00,  0.000e+00,  0.000e+00, -1.332e-15,
         2.429e-16,  2.810e-16],
       [ 9.714e-17, -6.245e-17,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  7.494e-16,
        -6.245e-16,  1.145e-16],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  1.665e-16, -9.714e-17,  7.633e-16,
         2.831e-15, -2.276e-15],
       [ 0.000e+00,  0.000e+00,  0.000e+00, -1.110e-16,  0.000e+00,  2.776e-17, -2.887e-15,
         5.690e-16,  8.882e-16],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  9.714e-17, -6.245e-17,  0.000e+00,  1.832e-15,
        -1.582e-15,  2.845e-16],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         1.665e-16, -1.249e-16],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00, -1.110e-16,
         0.000e+00,  4.163e-17],


In [5]:
x.left_jacobian_inv()

SX(@1=0.989141, @2=-0.148329, @3=0.102506, @4=0.151671, @5=0.991647, @6=-0.0449883, @7=-0.0974941, @8=0.0550117, @9=0.995824, 
[[@1, @4, @7, 00, 00, 00, -0.0217683, 0.153349, -0.0949765], 
 [@2, @5, @8, 00, 00, 00, -0.146651, -0.0167448, 0.0600469], 
 [@3, @6, @9, 00, 00, 00, 0.105023, -0.0399531, -0.00837242], 
 [00, 00, 00, @1, @4, @7, -0.0468922, 0.310877, -0.234938], 
 [00, 00, 00, @2, @5, @8, -0.289123, -0.036842, 0.222606], 
 [00, 00, 00, @3, @6, @9, 0.265062, -0.177394, -0.0234327], 
 [00, 00, 00, 00, 00, 00, @1, @4, @7], 
 [00, 00, 00, 00, 00, 00, @2, @5, @8], 
 [00, 00, 00, 00, 00, 00, @3, @6, @9]])

In [6]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(x.left_jacobian()) @ ca.DM(x.left_jacobian_inv()))

array([[ 1.00000e+00, -9.62772e-17,  2.77556e-17,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.21431e-17,  3.55618e-17, -2.77556e-17],
       [ 8.50015e-17,  1.00000e+00, -8.32667e-17,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  2.73219e-17, -6.93889e-18],
       [-6.93889e-17, -2.08167e-17,  1.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.84314e-17, -1.95156e-18,  1.04083e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00, -9.62772e-17,  2.77556e-17,  4.16334e-17,  6.07153e-17, -5.55112e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  8.50015e-17,  1.00000e+00, -8.32667e-17,  3.46945e-18,  4.07660e-17, -5.55112e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -6.93889e-17, -2.08167e-17,  1.00000e+00,  3.90313e-17, -3.64292e-17,  1.38778e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00, -9.62772e-17,  2.77556e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00

In [8]:
x.right_jacobian()

SX(@1=0.978484, @2=-0.144948, @3=0.103804, @4=0.151568, @5=0.98345, @6=-0.0394891, @7=-0.0938736, @8=0.0593496, @9=0.991725, 
[[@1, @4, @7, 00, 00, 00, -0.0427297, 0.151364, -0.0866664], 
 [@2, @5, @8, 00, 00, 00, -0.138217, -0.032869, 0.067985], 
 [@3, @6, @9, 00, 00, 00, 0.106388, -0.0285422, -0.0164345], 
 [00, 00, 00, @1, @4, @7, -0.0919934, 0.31, -0.212181], 
 [00, 00, 00, @2, @5, @8, -0.267181, -0.0722919, 0.239404], 
 [00, 00, 00, @3, @6, @9, 0.271445, -0.150667, -0.0460762], 
 [00, 00, 00, 00, 00, 00, @1, @4, @7], 
 [00, 00, 00, 00, 00, 00, @2, @5, @8], 
 [00, 00, 00, 00, 00, 00, @3, @6, @9]])

In [9]:
x.right_jacobian_inv()

SX(@1=0.989141, @2=0.151671, @3=-0.0974941, @4=-0.148329, @5=0.991647, @6=0.0550117, @7=0.102506, @8=-0.0449883, @9=0.995824, 
[[@1, @4, @7, 00, 00, 00, -0.0217683, -0.146651, 0.105023], 
 [@2, @5, @8, 00, 00, 00, 0.153349, -0.0167448, -0.0399531], 
 [@3, @6, @9, 00, 00, 00, -0.0949765, 0.0600469, -0.00837242], 
 [00, 00, 00, @1, @4, @7, -0.0468922, -0.289123, 0.265062], 
 [00, 00, 00, @2, @5, @8, 0.310877, -0.036842, -0.177394], 
 [00, 00, 00, @3, @6, @9, -0.234938, 0.222606, -0.0234327], 
 [00, 00, 00, 00, 00, 00, @1, @4, @7], 
 [00, 00, 00, 00, 00, 00, @2, @5, @8], 
 [00, 00, 00, 00, 00, 00, @3, @6, @9]])

In [10]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(x.right_jacobian()) @ ca.DM(x.right_jacobian_inv()))

array([[ 1.00000e+00,  4.85723e-17, -6.93889e-17,  0.00000e+00,  0.00000e+00,  0.00000e+00,  2.25514e-17, -4.68375e-17,  1.38778e-17],
       [-1.04951e-16,  1.00000e+00, -2.08167e-17,  0.00000e+00,  0.00000e+00,  0.00000e+00, -1.73472e-18,  1.08420e-17, -1.38778e-17],
       [ 4.16334e-17, -7.63278e-17,  1.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -6.72205e-18, -2.38524e-18,  1.04083e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00,  4.85723e-17, -6.93889e-17,  3.81639e-17, -6.76542e-17,  5.55112e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -1.04951e-16,  1.00000e+00, -2.08167e-17,  3.46945e-17,  3.12250e-17, -2.77556e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  4.16334e-17, -7.63278e-17,  1.00000e+00, -7.80626e-17,  1.51788e-17,  2.77556e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00,  4.85723e-17, -6.93889e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00

In [5]:
import math

In [6]:
def derive_Ul_inv_series(n):
    x = lie.se23.elem(ca.SX.sym("x", 9))
    X = x.ad()
    Ul_inv = ca.SX.zeros(9, 9)
    # print(ca.mpower(X, 0))
    for k in range(n):
        Xp = ca.mpower(X, k)
        # print(k)
        # print(Xp.sparsity().spy())
        Ul_inv += Xp / math.factorial(k + 1)
    Ul_inv = ca.sparsify(Ul_inv)
    return ca.Function("Ul_inv_series", [x.param], [Ul_inv])


Ul_inv_series = derive_Ul_inv_series(10)

In [8]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(x.left_jacobian()) - ca.DM(Ul_inv_series(x.param)))

array([[-1.24989e-12,  2.25903e-13,  2.66065e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00, -1.24983e-11,  2.29233e-12,  2.63782e-12],
       [ 1.58734e-13, -9.61453e-13,  5.88092e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.55320e-12, -9.61405e-12,  5.89162e-12],
       [ 3.10835e-13,  5.65707e-13, -4.80838e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  3.13059e-12,  5.64523e-12, -4.80702e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -1.24989e-12,  2.25903e-13,  2.66065e-13, -2.82378e-11,  5.60102e-12,  6.43652e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.58734e-13, -9.61453e-13,  5.88092e-13,  3.93069e-12, -2.18101e-11,  1.34444e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  3.10835e-13,  5.65707e-13, -4.80838e-13,  7.57250e-12,  1.28431e-11, -1.14820e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -1.24989e-12,  2.25903e-13,  2.66065e-13],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00

In [9]:
%%timeit
x.left_jacobian

31.5 ns ± 0.138 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
%%timeit
Ul_inv_series(x.param)

888 µs ± 543 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
def left_jacobian_barfoot():
    x = ca.SX.sym("x", 9)
    v = x[:3]
    a = x[3:6]
    omega = x[6:]
    m = ca.SX.sym("m", 3)
    o = ca.SX.sym("o", 3)
    M = lie.so3.elem(m).ad()
    O = lie.so3.elem(o).ad()
    O_sq = O @ O
    theta = ca.norm_2(o)
    # A = so3.elem(a.param).ad()
    # V = so3.elem(v.param).ad()
    c_theta = ca.cos(theta)
    s_theta = ca.sin(theta)

    Coeff = ca.if_else(
        ca.fabs(theta) > 1e-3,
        ca.vertcat(
            (1 - c_theta) / (theta**2),  # C0
            (theta - s_theta) / (theta**3),  # C1
            (theta**2 + 2 * c_theta - 2) / (2 * theta**4),  # C2
            (theta * c_theta + 2 * theta - 3 * s_theta) / (2 * theta**5),  # C3
            (theta**2 + theta * s_theta + 4 * c_theta - 4) / (2 * theta**6),  # C4
            (2 - 2 * c_theta - theta * s_theta) / (2 * theta**4),  # C5
        ),
        ca.vertcat(
            1 / 2 - theta**2 / 24 + theta**4 / 720,
            1 / 6 - theta**2 / 120 + theta**4 / 5040,
            1 / 24 - theta**2 / 720 + theta**4 / 40320,
            1 / 120 - theta**2 / 2520 + theta**4 / 120960,
            1 / 720 - theta**2 / 20160 + theta**4 / 1209600,
            1 / 24 - theta**2 / 360 + theta**4 / 134400,
        ),
    )

    R = ca.SX.eye(3) + Coeff[0] * O + Coeff[1] * O_sq

    C = M / 2
    C += Coeff[1] * (O @ M + M @ O + O @ M @ O)
    C += Coeff[2] * (O_sq @ M + M @ O_sq - 3 * O @ M @ O)
    C += Coeff[3] * (O @ M @ O_sq + O_sq @ M @ O)

    f_C = ca.Function("f_C", [m, o], [C])
    f_R = ca.Function("f_R", [o], [R])

    C_A = f_C(a, omega)
    C_V = f_C(v, omega)
    R = f_R(omega)
    Z = ca.SX.zeros(3, 3)
    Jl = ca.sparsify(
        ca.vertcat(ca.horzcat(R, Z, C_V), ca.horzcat(Z, R, C_A), ca.horzcat(Z, Z, R))
    )

    return ca.Function("J_barfoot", [x], [Jl])


Jcl = left_jacobian_barfoot()

In [14]:
Jcl(x.param)

SX(@1=0.978484, @2=0.151568, @3=-0.0938736, @4=-0.144948, @5=0.98345, @6=0.0593496, @7=0.103804, @8=-0.0394891, @9=0.991725, 
[[@1, @4, @7, 00, 00, 00, -0.0427297, -0.138217, 0.106388], 
 [@2, @5, @8, 00, 00, 00, 0.151364, -0.032869, -0.0285422], 
 [@3, @6, @9, 00, 00, 00, -0.0866664, 0.067985, -0.0164345], 
 [00, 00, 00, @1, @4, @7, -0.0919934, -0.267181, 0.271445], 
 [00, 00, 00, @2, @5, @8, 0.31, -0.0722919, -0.150667], 
 [00, 00, 00, @3, @6, @9, -0.212181, 0.239404, -0.0460762], 
 [00, 00, 00, 00, 00, 00, @1, @4, @7], 
 [00, 00, 00, 00, 00, 00, @2, @5, @8], 
 [00, 00, 00, 00, 00, 00, @3, @6, @9]])

In [21]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(x.left_jacobian()) - ca.DM(Jcl(x.param)))

array([[ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -7.99361e-15,  5.32907e-15],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  7.77156e-15,  0.00000e+00, -2.66454e-15],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -5.32907e-15,  2.77556e-15,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -1.77636e-14,  1.24345e-14],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.86517e-14,  0.00000e+00, -5.77316e-15],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -1.19904e-14,  6.66134e-15,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00

In [26]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(Jcl(x.param)) - ca.DM(Ul_inv_series(x.param)))

array([[-9.32143e-13,  3.76643e-14,  5.93969e-15,  0.00000e+00,  0.00000e+00,  0.00000e+00, -9.37255e-11,  5.31553e-12,  2.37743e-12],
       [-8.96505e-15, -6.46150e-13,  4.31048e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.50102e-13, -6.50880e-11,  4.34137e-11],
       [ 3.70814e-14,  4.29495e-13, -2.87548e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  5.82101e-12,  4.31021e-11, -2.90398e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -9.32143e-13,  3.76643e-14,  5.93969e-15, -2.02928e-10,  1.41598e-11,  9.00791e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -8.96505e-15, -6.46150e-13,  4.31048e-13,  3.03935e-12, -1.39202e-10,  9.49660e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  3.70814e-14,  4.29495e-13, -2.87548e-13,  1.65765e-11,  9.40066e-11, -6.51243e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -9.32143e-13,  3.76643e-14,  5.93969e-15],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00

In [25]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(x.left_jacobian()) - ca.DM(Ul_inv_series(x.param)))

array([[-9.32143e-13,  3.76643e-14,  5.93969e-15,  0.00000e+00,  0.00000e+00,  0.00000e+00, -9.37255e-11,  5.31641e-12,  2.37699e-12],
       [-8.96505e-15, -6.46150e-13,  4.31048e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  1.49436e-13, -6.50880e-11,  4.34137e-11],
       [ 3.70814e-14,  4.29495e-13, -2.87548e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  5.82157e-12,  4.31020e-11, -2.90398e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -9.32143e-13,  3.76643e-14,  5.93969e-15, -2.02928e-10,  1.41620e-11,  9.00657e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -8.96505e-15, -6.46150e-13,  4.31048e-13,  3.03757e-12, -1.39202e-10,  9.49663e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  3.70814e-14,  4.29495e-13, -2.87548e-13,  1.65774e-11,  9.40066e-11, -6.51243e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -9.32143e-13,  3.76643e-14,  5.93969e-15],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00

In [16]:
x = lie.so3.elem(ca.vertcat(0.01, 0.2, 0.3))

In [17]:
x.left_jacobian() @ x.left_jacobian_inv()

SX(@1=1, 
[[@1, 6.59733e-18, 0], 
 [-1.0842e-18, @1, -1.73472e-17], 
 [1.38778e-17, -1.77267e-17, @1]])

In [18]:
np.array(
    ca.DM(x.right_jacobian()) @ scipy.linalg.expm(ca.DM(x.ad()))
    - ca.DM(x.left_jacobian())
)

array([[ 0.000e+00,  0.000e+00,  5.551e-17],
       [ 0.000e+00,  0.000e+00,  0.000e+00],
       [-1.388e-17,  1.735e-18,  0.000e+00]])

In [5]:
x.left_jacobian()

SX(
[[0.978474, -0.14805, 0.0994173], 
 [0.148712, 0.985081, 0.00498912], 
 [-0.0984238, 0.0148812, 0.99336]])

In [6]:
x.left_jacobian_inv()

SX(
[[0.989143, 0.150167, -0.0997495], 
 [-0.149833, 0.992475, 0.0100109], 
 [0.100251, 1.08754e-05, 0.996651]])

In [7]:
x.right_jacobian() @ x.right_jacobian_inv()

SX(@1=1, 
[[@1, -6.93889e-18, 1.38778e-17], 
 [8.67362e-19, @1, -1.9082e-17], 
 [0, -1.73472e-17, @1]])

In [8]:
x.right_jacobian_inv()

SX(
[[0.989143, -0.149833, 0.100251], 
 [0.150167, 0.992475, 1.08754e-05], 
 [-0.0997495, 0.0100109, 0.996651]])

In [9]:
x.right_jacobian()

SX(
[[0.978474, 0.148712, -0.0984238], 
 [-0.14805, 0.985081, 0.0148812], 
 [0.0994173, 0.00498912, 0.99336]])

In [3]:
x = lie.se3.elem(ca.vertcat(0.4, 0.5, 0.6, 0.1, 0.2, 0.3))

In [6]:
lie.se3.right_Q(x.v_b, x.Omega)

SX(
[[-0.0919934, 0.31, -0.212181], 
 [-0.267181, -0.0722919, 0.239404], 
 [0.271445, -0.150667, -0.0460762]])

In [20]:
np.array(
    ca.DM(x.right_jacobian()) @ scipy.linalg.expm(ca.DM(x.ad()))
    - ca.DM(x.left_jacobian())
)

array([[ 0.000e+00,  1.665e-16, -1.249e-16,  7.633e-16,  2.942e-15, -2.276e-15],
       [-1.110e-16,  0.000e+00,  4.163e-17, -2.998e-15,  5.412e-16,  8.882e-16],
       [ 9.714e-17, -6.245e-17,  0.000e+00,  1.832e-15, -1.582e-15,  2.914e-16],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  1.665e-16, -9.714e-17],
       [ 0.000e+00,  0.000e+00,  0.000e+00, -1.110e-16,  0.000e+00,  2.776e-17],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  9.714e-17, -6.245e-17,  0.000e+00]])

In [45]:
x.right_jacobian()

SX(@1=0.978484, @2=-0.144948, @3=0.103804, @4=0.151568, @5=0.98345, @6=-0.0394891, @7=-0.0938736, @8=0.0593496, @9=0.991725, 
[[@1, @4, @7, -0.0919934, 0.31, -0.212181], 
 [@2, @5, @8, -0.267181, -0.0722919, 0.239404], 
 [@3, @6, @9, 0.271445, -0.150667, -0.0460762], 
 [00, 00, 00, @1, @4, @7], 
 [00, 00, 00, @2, @5, @8], 
 [00, 00, 00, @3, @6, @9]])

In [11]:
x.left_jacobian()

SX(@1=0.978484, @2=0.151568, @3=-0.0938736, @4=-0.144948, @5=0.98345, @6=0.0593496, @7=0.103804, @8=-0.0394891, @9=0.991725, 
[[@1, @4, @7, -0.0919934, -0.267181, 0.271445], 
 [@2, @5, @8, 0.31, -0.0722919, -0.150667], 
 [@3, @6, @9, -0.212181, 0.239404, -0.0460762], 
 [00, 00, 00, @1, @4, @7], 
 [00, 00, 00, @2, @5, @8], 
 [00, 00, 00, @3, @6, @9]])

In [8]:
x.right_jacobian_inv()

SX(@1=0.989141, @2=0.151671, @3=-0.0974941, @4=-0.148329, @5=0.991647, @6=0.0550117, @7=0.102506, @8=-0.0449883, @9=0.995824, 
[[@1, @4, @7, -0.0468922, -0.289123, 0.265062], 
 [@2, @5, @8, 0.310877, -0.036842, -0.177394], 
 [@3, @6, @9, -0.234938, 0.222606, -0.0234327], 
 [00, 00, 00, @1, @4, @7], 
 [00, 00, 00, @2, @5, @8], 
 [00, 00, 00, @3, @6, @9]])

In [10]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(x.right_jacobian()) @ ca.DM(x.right_jacobian_inv()))

array([[ 1.00000e+00,  4.85723e-17, -6.93889e-17,  3.81639e-17, -6.76542e-17,  5.55112e-17],
       [-1.04951e-16,  1.00000e+00, -2.08167e-17,  3.46945e-17,  3.12250e-17, -2.77556e-17],
       [ 4.16334e-17, -7.63278e-17,  1.00000e+00, -7.80626e-17,  1.51788e-17,  2.77556e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00,  4.85723e-17, -6.93889e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -1.04951e-16,  1.00000e+00, -2.08167e-17],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  4.16334e-17, -7.63278e-17,  1.00000e+00]])

In [46]:
x.right_Q(x.v_b, x.Omega)

SX(
[[-0.0919934, 0.31, -0.212181], 
 [-0.267181, -0.0722919, 0.239404], 
 [0.271445, -0.150667, -0.0460762]])

In [47]:
x.left_Q(x.v_b, x.Omega)

SX(
[[-0.0919934, -0.267181, 0.271445], 
 [0.31, -0.0722919, -0.150667], 
 [-0.212181, 0.239404, -0.0460762]])

In [4]:
C = scipy.linalg.expm(ca.DM(x.Omega.to_Matrix()))

In [8]:
(
    C.T @ lie.se3.left_Q(x.v_b, x.Omega)
    - C.T
    @ lie.so3.elem(x.Omega.left_jacobian() @ x.v_b.param).to_Matrix()
    @ x.Omega.left_jacobian()
)

SX(
[[-0.0919934, 0.31, -0.212181], 
 [-0.267181, -0.0722919, 0.239404], 
 [0.271445, -0.150667, -0.0460762]])

In [53]:
scipy.linalg.expm(ca.DM(x.ad()))

array([[ 0.936, -0.283,  0.21 , -0.273, -0.491,  0.556],
       [ 0.303,  0.951, -0.068,  0.618, -0.215, -0.248],
       [-0.181,  0.127,  0.975, -0.379,  0.512, -0.137],
       [ 0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 ],
       [ 0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068],
       [ 0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975]])

In [70]:
ca.DM(
    ca.vertcat(
        ca.horzcat(
            C, lie.so3.elem(x.Omega.left_jacobian() @ x.v_b.param).to_Matrix() @ C
        ),
        ca.horzcat(ca.SX.zeros(3, 3), C),
    )
)

DM(
[[0.935755, -0.283165, 0.210192, -0.273314, -0.490827, 0.555541], 
 [0.302933, 0.950581, -0.0680313, 0.61826, -0.214803, -0.248366], 
 [-0.18054, 0.127335, 0.97529, -0.379216, 0.512061, -0.137053], 
 [0, 0, 0, 0.935755, -0.283165, 0.210192], 
 [0, 0, 0, 0.302933, 0.950581, -0.0680313], 
 [0, 0, 0, -0.18054, 0.127335, 0.97529]])

In [74]:
ca.inv(
    ca.DM(
        ca.vertcat(
            ca.horzcat(
                C, C @ lie.so3.elem(x.Omega.right_jacobian() @ x.v_b.param).to_Matrix()
            ),
            ca.horzcat(ca.SX.zeros(3, 3), C),
        )
    )
)

DM(
[[0.935755, 0.302933, -0.18054, -0.273314, 0.61826, -0.379216], 
 [-0.283165, 0.950581, 0.127335, -0.490827, -0.214803, 0.512061], 
 [0.210192, -0.0680313, 0.97529, 0.555541, -0.248366, -0.137053], 
 [3.41736e-17, -4.82705e-17, 3.10843e-17, 0.935755, 0.302933, -0.18054], 
 [8.9873e-17, 2.57058e-18, -6.13118e-17, -0.283165, 0.950581, 0.127335], 
 [0, 1.22574e-17, -2.45149e-17, 0.210192, -0.0680313, 0.97529]])

In [80]:
lie.so3.elem(C @ x.Omega.right_jacobian() @ x.v_b.param).to_Matrix() @ C

SX(
[[-0.273314, -0.490827, 0.555541], 
 [0.61826, -0.214803, -0.248366], 
 [-0.379216, 0.512061, -0.137053]])

In [82]:
C @ lie.so3.elem(x.Omega.right_jacobian() @ x.v_b.param).to_Matrix()

SX(
[[-0.273314, -0.490827, 0.555541], 
 [0.61826, -0.214803, -0.248366], 
 [-0.379216, 0.512061, -0.137053]])

In [75]:
C.T

array([[ 0.936,  0.303, -0.181],
       [-0.283,  0.951,  0.127],
       [ 0.21 , -0.068,  0.975]])

In [76]:
C

array([[ 0.936, -0.283,  0.21 ],
       [ 0.303,  0.951, -0.068],
       [-0.181,  0.127,  0.975]])